In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [4]:
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

In [5]:
dataset = torchvision.datasets.MNIST(root='../../data/in',
                                    train=True,
                                    transform=transforms.ToTensor(),
                                    download=True)

Extracting ../../data/in/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/in/MNIST/raw


Extracting ../../data/in/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/in/MNIST/raw


Extracting ../../data/in/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/in/MNIST/raw




Extracting ../../data/in/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/in/MNIST/raw
Processing...
Done!


/home/kevinxu/.local/lib/python3.6/site-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
data_loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

In [9]:
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

In [10]:
model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [11]:
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1, image_size)
        x_reconst, mu, log_var = model(x)
        
        # Compute reconstruction loss and KL divergence
        reconst_loss = F.binary_cross_entropy(x_reconst, x, size_average=False)
        kl_div = - 0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
        
        # Backprop and optimise
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), 
                           reconst_loss.item(), kl_div.item()))
            
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size, z_dim).to(device)
        out = model.decode(z).view(-1, 1, 28, 28)
        save_image(out, os.path.join(sample_dir, 'sampled-{}.png'.format(epoch+1)))
        
        # Save the reconstructed images
        out, _, _ = model(x)
        x_concat = torch.cat([x.view(-1, 1, 28, 28), out.view(-1, 1, 28, 28)], dim=3)
        save_image(x_concat, os.path.join(sample_dir, 'reconst-{}.png'.format(epoch+1)))

/home/kevinxu/.local/lib/python3.6/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/kevinxu/.local/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/15], Step [10/469], Reconst Loss: 36089.8516, KL Div: 3667.4338
Epoch[1/15], Step [20/469], Reconst Loss: 29914.4297, KL Div: 904.3251
Epoch[1/15], Step [30/469], Reconst Loss: 27311.5957, KL Div: 1230.7915
Epoch[1/15], Step [40/469], Reconst Loss: 26694.3770, KL Div: 629.2594
Epoch[1/15], Step [50/469], Reconst Loss: 27225.7090, KL Div: 708.2644
Epoch[1/15], Step [60/469], Reconst Loss: 25336.0352, KL Div: 785.5221
Epoch[1/15], Step [70/469], Reconst Loss: 25095.7578, KL Div: 863.5602
Epoch[1/15], Step [80/469], Reconst Loss: 24490.9883, KL Div: 976.2679
Epoch[1/15], Step [90/469], Reconst Loss: 23837.1094, KL Div: 1020.4596
Epoch[1/15], Step [100/469], Reconst Loss: 22557.2734, KL Div: 1315.6624
Epoch[1/15], Step [110/469], Reconst Loss: 21732.7930, KL Div: 1472.5048
Epoch[1/15], Step [120/469], Reconst Loss: 20458.5234, KL Div: 1691.8617
Epoch[1/15], Step [130/469], Reconst Loss: 20765.3652, KL Div: 1741.7332
Epoch[1/15], Step [140/469], Reconst Loss: 19112.7422, KL Div: 174